In [1]:
import numpy as np
import gym
import tensorflow as tf

In [2]:
env = gym.make("CartPole-v0")

[2017-01-14 06:47:13,142] Making new env: CartPole-v0


In [3]:
H = 10
batch_size = 5
learning_rate = 1e-2
gamma = 0.99

D = 4

In [4]:
tf.reset_default_graph()
var_init = tf.contrib.layers.xavier_initializer()

observations = tf.placeholder(tf.float32, [None, D], name="input_x")

w1 = tf.get_variable("w1", shape=[D,H], initializer=var_init)
layer1 = tf.nn.relu(tf.matmul(observations, w1))

w2 = tf.get_variable("w2", shape=[H,1], initializer=var_init)
layer2 = tf.nn.relu(tf.matmul(layer1, w2))

probability = tf.nn.sigmoid(layer2)

In [5]:
tvars = tf.trainable_variables()
input_y = tf.placeholder(tf.float32, [None,1], name="input_y")
advantages = tf.placeholder(tf.float32, name="reward_signal")

In [6]:
loglik = tf.log(input_y*(input_y - probability) + (1 - input_y)*(input_y + probability))
loss = -tf.reduce_mean(loglik * advantages)
newGrads = tf.gradients(loss, tvars)

In [9]:
adam = tf.train.AdamOptimizer(learning_rate = learning_rate)

W1Grad = tf.placeholder(tf.float32, name="batch_grad1")
W2Grad = tf.placeholder(tf.float32, name="batch_grad2")

batchGrad = [W1Grad, W2Grad]
updateGrads = adam.apply_gradients(zip(batchGrad, tvars))

In [ ]:
def discount_rewards(r):
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r[t]